In [2]:
import sys
if not 'Autoformer' in sys.path[0]:
    sys.path[0] += '/Autoformer'


In [3]:
import argparse
import os
import torch
from exp.exp_main import Exp_Main
import random
import numpy as np
from utils.tools import dotdict

fix_seed = 2021 
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

2023-11-30:17:14:20,249 INFO     [utils.py:147] Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-11-30:17:14:20,250 INFO     [utils.py:159] NumExpr defaulting to 8 threads.


## Synthetic Data

### First experiment - prediction length 24
Set arguements for our model :<br>
Data - Synthetic dataset 1<br>
Model - Autoformer<br>
Frequency - hourly<br>
Features - univariate
Encoder length - 96<br>
Label length - 48<br>
Prediction horizon - 24<br>


In [4]:
# args = parser.parse_args()
args = dotdict()
args.target = 'TARGET'
args.des = 'test'
args.dropout = 0.05
args.num_workers = 10
args.freq = 'h'
args.checkpoints = './checkpoints/'
args.bucket_size = 4
args.n_hashes = 4
args.is_trainging = True
args.root_path = './SYNTHDataset'
args.data_path ='SYNTHh1.csv' 
args.model_id='Synth1_96_24'
args.model = 'Autoformer'
args.data = 'Synth1'
args.features = 'S' #univariate
args.seq_len = 96
args.label_len = 48
args.pred_len = 24
args.e_layers = 2
args.d_layers = 1
args.n_heads = 8
args.factor = 1
args.enc_in = 1
args.dec_in =1
args.c_out = 1
args.d_model = 512
args.des = 'Exp'
args.itr = 1
args.d_ff = 2048
args.moving_avg = 25
args.factor = 1
args.distil = True
args.output_attention = False
args.patience= 3
args.learning_rate = 0.0001
args.batch_size = 32 # batch size 32 as in the paper
args.embed = 'timeF'
args.activation = 'gelu'
args.use_amp = False
args.loss = 'mse'
args.train_epochs = 10 # epoch size is 10 as in the paper


# GPU 
args.gpu = 0
args.lradj = 'type1'
args.devices = '0,1,2,3'
args.use_multi_gpu = False
args.use_gpu = True if torch.cuda.is_available() else False

if args.use_gpu and args.use_multi_gpu:
     args.devices = args.devices.replace(' ', '')
     device_ids = args.devices.split(',')
     args.device_ids = [int(id_) for id_ in device_ids]
     args.gpu = args.device_ids[0]

print('Args in experiment:')
print(args)

Exp = Exp_Main



Args in experiment:
{'target': 'TARGET', 'des': 'Exp', 'dropout': 0.05, 'num_workers': 10, 'freq': 'h', 'checkpoints': './checkpoints/', 'bucket_size': 4, 'n_hashes': 4, 'is_trainging': True, 'root_path': './SYNTHDataset', 'data_path': 'SYNTHh1.csv', 'model_id': 'Synth1_96_24', 'model': 'Autoformer', 'data': 'Synth1', 'features': 'S', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'e_layers': 2, 'd_layers': 1, 'n_heads': 8, 'factor': 1, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'd_model': 512, 'itr': 1, 'd_ff': 2048, 'moving_avg': 25, 'distil': True, 'output_attention': False, 'patience': 3, 'learning_rate': 0.0001, 'batch_size': 32, 'embed': 'timeF', 'activation': 'gelu', 'use_amp': False, 'loss': 'mse', 'train_epochs': 10, 'gpu': 0, 'lradj': 'type1', 'devices': '0,1,2,3', 'use_multi_gpu': False, 'use_gpu': True}


Train the model

In [5]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

for ii in range(args.itr):
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp_synth24 = Exp(args)  # set experiments
    print(1)
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp_synth24.train(setting)
    print(2)
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    mae_synth24 , mse_synth24 , y_pred_synth24, y_true_synth24, first_batch_test_synth24 = exp_synth24.test(setting) 
    torch.cuda.empty_cache()
    print(3)



Use GPU: cuda:0
1
>>>>>>>start training : Synth1_96_24_Autoformer_Synth1_ftS_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.3143713
	speed: 0.0443s/iter; left time: 113.4498s
	iters: 200, epoch: 1 | loss: 0.1954803
	speed: 0.0360s/iter; left time: 88.5610s
Epoch: 1 cost time: 10.460726022720337
Epoch: 1, Steps: 266 | Train Loss: 0.3389484 Vali Loss: 0.1933758 Test Loss: 0.1858957
Validation loss decreased (inf --> 0.193376).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1122024
	speed: 0.1666s/iter; left time: 382.2853s
	iters: 200, epoch: 2 | loss: 0.0861645
	speed: 0.0416s/iter; left time: 91.2994s
Epoch: 2 cost time: 11.356830358505249
Epoch: 2, Steps: 266 | Train Loss: 0.1275285 Vali Loss: 0.1341929 Test Loss: 0.1322345
Validation loss decreased (0.193376 --> 0.134193).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 

In [6]:
#Check the shape of the predictions and compare to real observations
print (y_pred_synth24.shape, y_true_synth24.shape)


(2857, 24, 1) (2857, 24, 1)


We have 2857 sequences of length 24, which is the prediction length we set.

In [7]:
#Shape of the the encoder and of the output in a single batch
print (first_batch_test_synth24['batch_x'].shape , first_batch_test_synth24['batch_y'].shape)

torch.Size([32, 96, 1]) (32, 24, 1)


In [8]:
#Shape of the marked data encoder and of the output in a single batch. 
#Each observation is assigned 4 marks - minute,hour,day of the week and day of the year.
print (first_batch_test_synth24['batch_x_mark'].shape , first_batch_test_synth24['batch_y_mark'].shape)

torch.Size([32, 96, 4]) torch.Size([32, 72, 4])


### Second experiment - prediction length 168
Set arguements for our model :<br>
Data - Synthetic dataset 1<br>
Model - Autoformer<br>
Frequency - hourly<br>
Features - univariate
Encoder length - 96<br>
Label length - 48<br>
Prediction horizon - 168<br>


In [9]:
#All we have to do is change the prediction length arguement, all other arguements stay the same
args.pred_len = 168
args.model_id='Synth1_96_168'

In [10]:


for ii in range(args.itr):
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp_synth168 = Exp(args)  # set experiments
    print(1)
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp_synth168.train(setting)
    print(2)
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    mae_synth168 , mse_synth168 , y_pred_synth168, y_true_synth168, first_batch_test_synth168 = exp_synth168.test(setting) 
    torch.cuda.empty_cache()
    print(3)



Use GPU: cuda:0
1
>>>>>>>start training : Synth1_96_168_Autoformer_Synth1_ftS_sl96_ll48_pl168_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8377
val 2713
test 2713
	iters: 100, epoch: 1 | loss: 0.8597628
	speed: 0.0522s/iter; left time: 131.0589s
	iters: 200, epoch: 1 | loss: 0.7179357
	speed: 0.0495s/iter; left time: 119.2551s
Epoch: 1 cost time: 13.303253889083862
Epoch: 1, Steps: 261 | Train Loss: 0.7818099 Vali Loss: 0.7682859 Test Loss: 0.8063470
Validation loss decreased (inf --> 0.768286).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.4742875
	speed: 0.1815s/iter; left time: 408.2964s
	iters: 200, epoch: 2 | loss: 0.4865830
	speed: 0.0490s/iter; left time: 105.3118s
Epoch: 2 cost time: 12.991724252700806
Epoch: 2, Steps: 261 | Train Loss: 0.4661411 Vali Loss: 0.6975989 Test Loss: 0.7464719
Validation loss decreased (0.768286 --> 0.697599).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch

### Third experiment - prediction length 720
Set arguements for our model :<br>
Data - Synthetic dataset 1<br>
Model - Autoformer<br>
Frequency - hourly<br>
Features - univariate
Encoder length - 96<br>
Label length - 48<br>
Prediction horizon - 720<br>


In [11]:
args.pred_len = 720
args.model_id='Synth1_96_720'

In [12]:


for ii in range(args.itr):
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp_synth720 = Exp(args)  # set experiments
    print(1)
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp_synth720.train(setting)
    print(2)
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    mae_synth720 , mse_synth720 , y_pred_synth720, y_true_synth720, first_batch_test_synth720 = exp_synth720.test(setting) 
    torch.cuda.empty_cache()
    print(3)



Use GPU: cuda:0
1
>>>>>>>start training : Synth1_96_720_Autoformer_Synth1_ftS_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 7825
val 2161
test 2161
	iters: 100, epoch: 1 | loss: 0.9376960
	speed: 0.1235s/iter; left time: 289.1599s
	iters: 200, epoch: 1 | loss: 0.6360489
	speed: 0.1220s/iter; left time: 273.3708s
Epoch: 1 cost time: 29.985873222351074
Epoch: 1, Steps: 244 | Train Loss: 0.8518100 Vali Loss: 0.9108309 Test Loss: 1.1545044
Validation loss decreased (inf --> 0.910831).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.7372466
	speed: 0.3158s/iter; left time: 662.2377s
	iters: 200, epoch: 2 | loss: 0.5897607
	speed: 0.1139s/iter; left time: 227.4784s
Epoch: 2 cost time: 28.314762353897095
Epoch: 2, Steps: 244 | Train Loss: 0.6596435 Vali Loss: 0.8983436 Test Loss: 1.1180611
Validation loss decreased (0.910831 --> 0.898344).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch

## Wind data

Set arguements for our model :<br>
Data - Wind dataset - Germany data<br>
Model - Autoformer<br>
Frequency - hourly<br>
Features - univariate<br>
Encoder length - 96<br>
Label length - 48<br>
Prediction horizon - 24<br>

In [14]:
args.root_path = './WINDataset'
args.data_path ='DEWINDh_small.csv' 
args.model_id='Windh1_96_24'
args.data = 'Windh1'
args.model = 'Autoformer'
args.freq = 'h'
args.features = 'S' #univariate
args.seq_len = 96
args.label_len = 48
args.pred_len = 24


In [15]:
for ii in range(args.itr):
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp_wind24 = Exp(args)  # set experiments
    print(1)
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp_wind24.train(setting)
    print(2)
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    mae_wind24 , mse_wind24 , y_pred_wind24, y_true_wind24, first_batch_test_wind24 = exp_wind24.test(setting) 
    torch.cuda.empty_cache()
    print(3)

Use GPU: cuda:0
1
>>>>>>>start training : Windh1_96_24_Autoformer_Windh1_ftS_sl96_ll48_pl24_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.2013479
	speed: 0.0354s/iter; left time: 90.7487s
	iters: 200, epoch: 1 | loss: 0.1326992
	speed: 0.0335s/iter; left time: 82.4593s
Epoch: 1 cost time: 9.12367057800293
Epoch: 1, Steps: 266 | Train Loss: 0.2505927 Vali Loss: 0.2473148 Test Loss: 0.2902733
Validation loss decreased (inf --> 0.247315).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.1633435
	speed: 0.1370s/iter; left time: 314.3430s
	iters: 200, epoch: 2 | loss: 0.1217971
	speed: 0.0317s/iter; left time: 69.6385s
Epoch: 2 cost time: 8.699025630950928
Epoch: 2, Steps: 266 | Train Loss: 0.1608405 Vali Loss: 0.2154517 Test Loss: 0.2235872
Validation loss decreased (0.247315 --> 0.215452).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | lo

### Second experiment - prediction length 168
Set arguements for our model :<br>
Data - Wind dataset 1<br>
Model - Autoformer<br>
Frequency - hourly<br>
Feature - univariate<br>
Encoder length - 96<br>
Label length - 48<br>
Prediction horizon - 168<br>


In [16]:
args.model_id='Windh1_96_168'
args.pred_len = 168


In [17]:
for ii in range(args.itr):
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp_wind168 = Exp(args)  # set experiments
    print(1)
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp_wind168.train(setting)
    print(2)
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    mae_wind168 , mse_wind168 , y_pred_wind168, y_true_wind168, first_batch_test_wind168 = exp_wind168.test(setting) 
    torch.cuda.empty_cache()
    print(3)

Use GPU: cuda:0
1
>>>>>>>start training : Windh1_96_168_Autoformer_Windh1_ftS_sl96_ll48_pl168_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8377
val 2713
test 2713
	iters: 100, epoch: 1 | loss: 0.2818532
	speed: 0.0506s/iter; left time: 126.9818s
	iters: 200, epoch: 1 | loss: 0.2483424
	speed: 0.0481s/iter; left time: 115.9414s
Epoch: 1 cost time: 12.970085620880127
Epoch: 1, Steps: 261 | Train Loss: 0.3334505 Vali Loss: 0.2943013 Test Loss: 0.2996873
Validation loss decreased (inf --> 0.294301).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.2628716
	speed: 0.2056s/iter; left time: 462.6881s
	iters: 200, epoch: 2 | loss: 0.2037293
	speed: 0.0514s/iter; left time: 110.6053s
Epoch: 2 cost time: 13.780773401260376
Epoch: 2, Steps: 261 | Train Loss: 0.2522194 Vali Loss: 0.2668315 Test Loss: 0.2879464
Validation loss decreased (0.294301 --> 0.266832).  Saving model ...
Updating learning rate to 5e-05
	iters: 100, epoch

### Third experiment - prediction length 720
Set arguements for our model :<br>
Data - Wind dataset 1<br>
Model - Autoformer<br>
Frequency - hourly<br>
Feature - univariate<br>
Encoder length - 96<br>
Label length - 48<br>
Prediction horizon - 720<br>


In [18]:
args.model_id='Windh1_96_720'
args.pred_len = 720


In [19]:
for ii in range(args.itr):
    setting = '{}_{}_{}_ft{}_sl{}_ll{}_pl{}_dm{}_nh{}_el{}_dl{}_df{}_fc{}_eb{}_dt{}_{}_{}'.format(
        args.model_id,
        args.model,
        args.data,
        args.features,
        args.seq_len,
        args.label_len,
        args.pred_len,
        args.d_model,
        args.n_heads,
        args.e_layers,
        args.d_layers,
        args.d_ff,
        args.factor,
        args.embed,
        args.distil,
        args.des, ii)

    exp_wind720 = Exp(args)  # set experiments
    print(1)
    print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
    exp_wind720.train(setting)
    print(2)
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    mae_wind720 , mse_wind720 , y_pred_wind720, y_true_wind720, first_batch_test_wind720 = exp_wind720.test(setting) 
    torch.cuda.empty_cache()
    print(3)

Use GPU: cuda:0
1
>>>>>>>start training : Windh1_96_720_Autoformer_Windh1_ftS_sl96_ll48_pl720_dm512_nh8_el2_dl1_df2048_fc1_ebtimeF_dtTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 7825
val 2161
test 2161
	iters: 100, epoch: 1 | loss: 0.3441183
	speed: 0.1244s/iter; left time: 291.3119s
	iters: 200, epoch: 1 | loss: 0.3367561
	speed: 0.1188s/iter; left time: 266.2574s
Epoch: 1 cost time: 29.576159238815308
Epoch: 1, Steps: 244 | Train Loss: 0.3685992 Vali Loss: 0.3536319 Test Loss: 0.3061326
Validation loss decreased (inf --> 0.353632).  Saving model ...
Updating learning rate to 0.0001
	iters: 100, epoch: 2 | loss: 0.3035700
	speed: 0.3125s/iter; left time: 655.2985s
	iters: 200, epoch: 2 | loss: 0.2501293
	speed: 0.1279s/iter; left time: 255.3315s
Epoch: 2 cost time: 30.792705297470093
Epoch: 2, Steps: 244 | Train Loss: 0.2886758 Vali Loss: 0.3844312 Test Loss: 0.2870901
EarlyStopping counter: 1 out of 3
Updating learning rate to 5e-05
	iters: 100, epoch: 3 | loss: 0.2599574
	speed: 0.334